In [1]:
from typing import Optional
import fabric
from pathlib import Path
import re
import pandas as pd
import wandb
import random
import string
import logging

logging.basicConfig(level=logging.INFO)


def generate_model_name(prefix="model-", length=8):
    # Define the characters to choose from (alphanumeric)
    characters = string.ascii_lowercase + string.digits
    # Generate a random string of the specified length
    random_suffix = "".join(random.choice(characters) for _ in range(length))
    # Concatenate the prefix and the random suffix
    model_name = prefix + random_suffix
    return model_name


class LocalWandbApi(object):
    def __init__(
        self,
        project_dir: str,
        wandb_project: str,
        wandb_entity: str,
        ssh_host: Optional[str] = None,
    ):
        self.project_dir = project_dir
        self.ssh_host = ssh_host
        self.runner = fabric.Connection(ssh_host) if ssh_host else None
        self.wandb_dir = f"{self.project_dir}/detrex/wandb_output/wandb"
        self.folder_pat = re.compile(r"run-\d{8}_\d{6}-\w{7}")
        self.wandb_project = wandb_project
        self.wandb_entity = wandb_entity
        self.logger = logging.Logger("LocalWandbApi", level=logging.INFO)

    def list_runs(self) -> dict[str, Path]:
        """Return a dictionary run_id: run_folder."""
        output = self.runner.run(f"ls {self.wandb_dir}", hide=True).stdout
        folders = output.split("\n")
        runs = {
            run.split("-")[-1]: Path(f"{self.wandb_dir}/{run}")
            for run in folders
            if self.folder_pat.match(run)
        }
        return runs

    def get_run_folder(self, run_id: str) -> Path:
        """Return the folder of a run."""
        # Example: run-20240805_162409-67c1veuz
        runs = self.list_runs()
        return runs[run_id]

    def get_run_logs_raw(self, run_id: str) -> str:
        """Return the raw logs of a run."""
        run_folder = self.get_run_folder(run_id)
        output = self.runner.run(f"cat {run_folder}/files/output.log", hide=True).stdout
        files = output.split("\n")
        return files

    def get_run_logs(self, run_id: str) -> pd.DataFrame:
        """Return a DataFrame with columns date, time, logger, message."""
        logs = self.get_run_logs_raw(run_id)
        pat = re.compile(r"^\[(\d{2}/\d{2}) (\d{2}:\d{2}:\d{2}) (.*)\]: (.+)")
        data = []
        columns = ["date", "time", "logger", "message"]
        for log in logs:
            # filter rows that do not contain logger prefix
            # Examples:
            # [08/05 18:18:58 d2.evaluation.evaluator]: asdfsadfj asdfas
            # [08/05 18:20:13 d2.evaluation.fast_eval_api]: asdfsdf asdfasd
            match = pat.match(log)
            if match:
                data.append(match.groups())
        df = pd.DataFrame(data, columns=columns)
        return df

    def log_checkpoint(
        self, run_id: str, ckpt_path: Path, artifact_name: Optional[str] = None
    ):
        """Log a checkpoint to a run."""
        self.logger.info(f"Logging checkpoint {ckpt_path} to run {run_id}")
        artifact_name = artifact_name or generate_model_name()
        with wandb.init(
            entity=self.wandb_entity,
            project=self.wandb_project,
            id=run_id,
            resume="allow",
        ) as run:
            self.logger.info(f"Logging artifact {artifact_name}")
            artifact = wandb.Artifact(artifact_name, "model")
            artifact.add_file(ckpt_path)
            run.log_artifact(artifact)

/Users/dgcnz/development/amsterdam/edge/.venv/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/dgcnz/development/amsterdam/edge/.venv/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
api = LocalWandbApi(
    "~/development/edge",
    ssh_host="amsterdam",
    wandb_entity="ams-edge",
    wandb_project="detrex",
)

In [3]:
ckpt_path = "../artifacts/model_final.pth"
run_id = "qftlioc0" # last run
api.log_checkpoint(run_id, ckpt_path)

hi


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: diego-canez-ildefonso (ams-edge). Use `wandb login --relogin` to force relogin


bbox/AP,51.94385
bbox/AP-airplane,77.44832
bbox/AP-apple,29.53469
bbox/AP-backpack,22.86667
bbox/AP-banana,35.20076
bbox/AP-baseball bat,48.81458
bbox/AP-baseball glove,46.6375
bbox/AP-bear,84.32996
bbox/AP-bed,61.56205
bbox/AP-bench,37.76952
bbox/AP-bicycle,40.20304
